In [273]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../..")
from data.dataset import DATASET as mainDf
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, RobustScaler

In [309]:
# write the necessary functions
# df = mainDf.loc[mainDf['risk1'].notnull()]
# t = np.where(np.isnan(x))

def getRowsForRisks(df, columns):
    partialDf = []
    for column in columns:
        data = df.loc[df[column].notnull()]
        partialDf.append(data)

    return partialDf

def preprocess(data, risks):
    data = data.drop(columns=['country','city','country_code','c40'])
    data = data.drop(columns=risks)

    return data

def getImportantFeatures(df, label, risks):
    y = df.loc[:, label]
    x = df.loc[:, df.columns != label]

    #risks.remove(label)
    #print(risks)
    
    x = x.drop(columns=['country','city','country_code','c40'])
    cols = [col for col in x.columns if col.lower()[:4] != 'risk']
    x = x[cols]

    #l = x.columns[x.isna().any()].tolist()
    #print(l)

    var_num = x.shape[1]
    var_num = int((var_num*15)/100)
    print("Picked variable number:",var_num)

    # Applying select K-best
    bestFeatures = SelectKBest(score_func=f_classif, k=var_num)
    fit = bestFeatures.fit(x,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(x.columns)

    #Concat two dataframes for better visualization
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']
    print(featureScores.nlargest(var_num,'Score'))  #print 15% of the total features according to the score features

def getArrayOfFeatures(data, name):
    arr = data.columns.values
    return [s for s in arr if (name in s)]

def generateCrossFeat(data, arr):
    poly = PolynomialFeatures()
    crossed_feats = poly.fit_transform(data[arr].values)

    #Convert to Pandas DataFrame and merge to original dataset
    crossed_df = pd.DataFrame(crossed_feats)
    #print(crossed_df.shape)
    return crossed_df
    

In [ ]:
#mainDf['Density'] = mainDf['']

In [258]:
display(mainDf)
#print(mainDf.info)

,city,latitude,longitude,population,country_code,c40,risk0,risk1,risk2,risk3,...,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP),country
0,Aalborg,57.0337,9.9166,122219.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
1,Aarhus,56.1572,10.2107,237551.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
2,Copenhagen,55.6786,12.5635,1085000.0,DNK,False,NaN,2.0,NaN,2.0,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
3,Esbjerg,55.4670,8.4500,72205.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
4,Frederikshavn,57.4337,10.5333,24103.0,DNK,False,NaN,2.0,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,Teaoraereke,1.3333,173.0000,5105.0,KIR,False,NaN,NaN,NaN,NaN,...,110.288104,80.302831,68.505925,2.532741,-1779.609434,66.9,0.000000,414.127038,87.3,Kiribati
1207,Trindade,0.3000,6.6667,6636.0,STP,False,NaN,NaN,NaN,NaN,...,17.162254,7.489641,1.876147,2.502891,10526.874919,97.1,10526.874919,197.499626,98.9,Sao Tome and Principe
1208,Victoria,-4.6236,55.4544,26450.0,SYC,False,NaN,NaN,NaN,NaN,...,120.471023,89.323035,69.243227,1.601611,-2157.596566,95.7,0.000000,142.089651,95.7,Seychelles
1209,Warsaw,52.2167,21.0333,1790658.0,POL,True,0.0,0.0,2.0,0.0,...,153.691550,43.570222,34.890966,0.600000,1412.266377,98.3,1594.069324,265.616741,99.3,Poland


In [281]:
#mainDf.columns.values

In [228]:
a = mainDf.columns.values.tolist()
#print(a)
risks = a[6:13]
risks_index = [i for i in range(6,14)]
print(risks)
print(risks_index)

['risk0', 'risk1', 'risk2', 'risk3', 'risk4', 'risk5', 'risk6']
[6, 7, 8, 9, 10, 11, 12, 13]


In [259]:
mylist = getRowsForRisks(mainDf, risks)
print(len(mylist))
for i in range(len(mylist)):
    print("Only",mylist[i].shape[0],"rows are available for risk",i)

7
Only 87 rows are available for risk 0
Only 148 rows are available for risk 1
Only 262 rows are available for risk 2
Only 184 rows are available for risk 3
Only 98 rows are available for risk 4
Only 65 rows are available for risk 5
Only 59 rows are available for risk 6


In [209]:
#display(mylist[0])

In [230]:
for i in range(len(risks)):
    print("For the", risks[i])
    getImportantFeatures(mylist[i], risks[i], risks)

For the risk0
Picked variable number: 21
                                                 Specs     Score
57   Gross domestic product (GDP), total (2017 PPP ...  4.032056
41             Population ages 65 and older (millions)  3.234697
52   Working poor at PPP$3.20 a day (% of total emp...  2.961307
27                         Inequality in education (%)  2.900850
4    Population with at least some secondary educat...  2.875645
3    Population with at least some secondary educat...  2.794997
17   Foreign direct investment, net inflows (% of GDP)  2.734514
5    Population with at least some secondary educat...  2.687594
8       Share of seats in parliament (% held by women)  2.518464
43                      Exports and imports (% of GDP)  2.435197
97     Mortality rate, under-5 (per 1,000 live births)  2.434818
54      Youth unemployment rate (female to male ratio)  2.378745
6              Mean years of schooling, female (years)  2.320540
29                     Mean years of schooling (y

In [ ]:
# Density features
# Try to get other algorithms

# PCA Part - using standard scaler

In [236]:
pca_df = preprocess(mainDf, risks)

val = pca_df.loc[:, pca_df.columns].values
print(val)

val = StandardScaler().fit_transform(val)
print(val.shape)

[[ 5.70337000e+01  9.91660000e+00  1.22219000e+05 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 [ 5.61572000e+01  1.02107000e+01  2.37551000e+05 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 [ 5.56786000e+01  1.25635000e+01  1.08500000e+06 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 ...
 [-4.62360000e+00  5.54544000e+01  2.64500000e+04 ...  0.00000000e+00
   1.42089651e+02  9.57000000e+01]
 [ 5.22167000e+01  2.10333000e+01  1.79065800e+06 ...  1.59406932e+03
   2.65616741e+02  9.93000000e+01]
 [ 3.13231000e+01  3.43440000e+01  3.20000000e+04 ...  1.76313456e+02
   7.90356138e+01  5.07000000e+01]]
(1211, 146)


In [237]:
np.mean(val), np.std(val)

(-1.5753580332000003e-17, 0.9999999999999999)

In [244]:
feat_cols = pca_df.columns.values.tolist()

normalized_val = pd.DataFrame(val, columns=feat_cols)
normalized_val.head()

,latitude,longitude,population,Population with at least some secondary education (% ages 25 and older),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)","Mean years of schooling, female (years)","Mean years of schooling, male (years)",Share of seats in parliament (% held by women),"Adolescent birth rate (births per 1,000 women ages 15-19)",...,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP)
0,1.467421,0.151965,-0.311815,0.874158,0.892489,0.843159,1.028015,0.90614,1.265226,-1.06401,...,0.197712,2.825792,2.028162,-0.104547,-1.089573,-0.400115,0.675537,-0.453103,-0.860845,0.575867
1,1.433283,0.156056,-0.275791,0.874158,0.892489,0.843159,1.028015,0.90614,1.265226,-1.06401,...,0.197712,2.825792,2.028162,-0.104547,-1.089573,-0.400115,0.675537,-0.453103,-0.860845,0.575867
2,1.414643,0.188786,-0.011093,0.874158,0.892489,0.843159,1.028015,0.90614,1.265226,-1.06401,...,0.197712,2.825792,2.028162,-0.104547,-1.089573,-0.400115,0.675537,-0.453103,-0.860845,0.575867
3,1.406401,0.131563,-0.327436,0.874158,0.892489,0.843159,1.028015,0.90614,1.265226,-1.06401,...,0.197712,2.825792,2.028162,-0.104547,-1.089573,-0.400115,0.675537,-0.453103,-0.860845,0.575867
4,1.483000,0.160544,-0.342461,0.874158,0.892489,0.843159,1.028015,0.90614,1.265226,-1.06401,...,0.197712,2.825792,2.028162,-0.104547,-1.089573,-0.400115,0.675537,-0.453103,-0.860845,0.575867


In [252]:
num_components = 10
pca_final = PCA(n_components=num_components)
pComponents_final = pca_final.fit_transform(val)
component_col = ['PC'+str(i+1) for i in range(num_components)]
print(component_col)

percentage_list = [element * 100 for element in pca_final.explained_variance_ratio_]
percentage_list = ['%.2f' % elem for elem in percentage_list]
print(percentage_list)

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10']
['34.82', '6.52', '5.80', '4.47', '3.82', '3.72', '3.20', '2.74', '2.37', '2.04']


In [253]:
pc_final_df = pd.DataFrame(data = pComponents_final, columns = component_col)
print(pc_final_df.shape)
pc_final_df.head()

(1211, 10)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,-9.540110,0.019157,0.477666,-2.337302,3.658370,-0.632142,-0.183594,3.254421,-1.015154,-1.577583
1,-9.537514,0.016874,0.479962,-2.332278,3.654878,-0.633276,-0.177372,3.256280,-1.015655,-1.578514
2,-9.535176,0.016018,0.474026,-2.299660,3.626128,-0.656317,-0.178641,3.264781,-1.028709,-1.579314
3,-9.536335,0.016155,0.484889,-2.337650,3.659678,-0.624048,-0.168553,3.254929,-1.012147,-1.578845
4,-9.540960,0.019347,0.476505,-2.341621,3.662356,-0.632251,-0.187811,3.253393,-1.013999,-1.577352


In [254]:
print('Explained variation percentage per principal component: {}'.format(percentage_list))
total_explained_percentage = (sum(pca_final.explained_variance_ratio_)*100)
print('Total percentage of the explained data by',pca_final.n_components,'components is: %.2f' %total_explained_percentage)
print('Percentage of the information that is lost for using',pca_final.n_components,'components is: %.2f' %(100-total_explained_percentage))

Explained variation percentage per principal component: ['34.82', '6.52', '5.80', '4.47', '3.82', '3.72', '3.20', '2.74', '2.37', '2.04']
Total percentage of the explained data by 10 components is: 69.51
Percentage of the information that is lost for using 10 components is: 30.49


In [255]:
mainDf_v2 = mainDf.merge(pc_final_df, left_index=True, right_index=True)
print(mainDf.shape)
print(mainDf_v2.shape)

(1211, 157)
(1211, 167)


## Doing the feature selection again after the addition of the Principal Components

In [256]:
mylist2 = getRowsForRisks(mainDf_v2, risks)
print(len(mylist2))
for i in range(len(mylist2)):
    print("Only",mylist2[i].shape[0],"rows are available for risk",i)

7
Only 87 rows are available for risk 0
Only 148 rows are available for risk 1
Only 262 rows are available for risk 2
Only 184 rows are available for risk 3
Only 98 rows are available for risk 4
Only 65 rows are available for risk 5
Only 59 rows are available for risk 6


In [261]:
for i in range(len(risks)):
    print("-- For the", risks[i],"--")
    getImportantFeatures(mylist2[i], risks[i], risks)

-- For the risk0 --
Picked variable number: 23
                                                 Specs     Score
57   Gross domestic product (GDP), total (2017 PPP ...  4.032056
41             Population ages 65 and older (millions)  3.234697
52   Working poor at PPP$3.20 a day (% of total emp...  2.961307
27                         Inequality in education (%)  2.900850
4    Population with at least some secondary educat...  2.875645
3    Population with at least some secondary educat...  2.794997
17   Foreign direct investment, net inflows (% of GDP)  2.734514
5    Population with at least some secondary educat...  2.687594
8       Share of seats in parliament (% held by women)  2.518464
43                      Exports and imports (% of GDP)  2.435197
97     Mortality rate, under-5 (per 1,000 live births)  2.434818
54      Youth unemployment rate (female to male ratio)  2.378745
155                                               PC10  2.372770
6              Mean years of schooling, fem

# PCA Part - using robust scaler

In [263]:
pca_df = preprocess(mainDf, risks)

val = pca_df.loc[:, pca_df.columns].values
print(val)

val = RobustScaler().fit_transform(val)
print(val.shape)

[[ 5.70337000e+01  9.91660000e+00  1.22219000e+05 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 [ 5.61572000e+01  1.02107000e+01  2.37551000e+05 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 [ 5.56786000e+01  1.25635000e+01  1.08500000e+06 ...  1.04670502e+03
   1.29285516e+02  1.00000000e+02]
 ...
 [-4.62360000e+00  5.54544000e+01  2.64500000e+04 ...  0.00000000e+00
   1.42089651e+02  9.57000000e+01]
 [ 5.22167000e+01  2.10333000e+01  1.79065800e+06 ...  1.59406932e+03
   2.65616741e+02  9.93000000e+01]
 [ 3.13231000e+01  3.43440000e+01  3.20000000e+04 ...  1.76313456e+02
   7.90356138e+01  5.07000000e+01]]
(1211, 146)


In [264]:
np.mean(val), np.std(val)

(0.14649351544290884, 2.799144105372385)

In [265]:
feat_cols = pca_df.columns.values.tolist()

normalized_val = pd.DataFrame(val, columns=feat_cols)
normalized_val.head()

,latitude,longitude,population,Population with at least some secondary education (% ages 25 and older),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)","Mean years of schooling, female (years)","Mean years of schooling, male (years)",Share of seats in parliament (% held by women),"Adolescent birth rate (births per 1,000 women ages 15-19)",...,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP)
0,0.963685,0.013189,-0.071419,0.435377,0.359522,0.424529,0.546993,0.495611,0.973721,-0.54967,...,4.632917,2.537586,10.047493,0.233602,-0.666667,-0.290289,0.190083,-0.373743,-0.384964,0.14
1,0.940453,0.015960,0.080529,0.435377,0.359522,0.424529,0.546993,0.495611,0.973721,-0.54967,...,4.632917,2.537586,10.047493,0.233602,-0.666667,-0.290289,0.190083,-0.373743,-0.384964,0.14
2,0.927767,0.038127,1.197029,0.435377,0.359522,0.424529,0.546993,0.495611,0.973721,-0.54967,...,4.632917,2.537586,10.047493,0.233602,-0.666667,-0.290289,0.190083,-0.373743,-0.384964,0.14
3,0.922158,-0.000628,-0.137312,0.435377,0.359522,0.424529,0.546993,0.495611,0.973721,-0.54967,...,4.632917,2.537586,10.047493,0.233602,-0.666667,-0.290289,0.190083,-0.373743,-0.384964,0.14
4,0.974288,0.018999,-0.200686,0.435377,0.359522,0.424529,0.546993,0.495611,0.973721,-0.54967,...,4.632917,2.537586,10.047493,0.233602,-0.666667,-0.290289,0.190083,-0.373743,-0.384964,0.14


In [266]:
num_components = 10
pca_final = PCA(n_components=num_components)
pComponents_final = pca_final.fit_transform(val)
component_col = ['PC'+str(i+1) for i in range(num_components)]
print(component_col)

percentage_list = [element * 100 for element in pca_final.explained_variance_ratio_]
percentage_list = ['%.2f' % elem for elem in percentage_list]
print(percentage_list)

['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10']
['34.06', '23.60', '14.98', '5.67', '3.73', '2.54', '1.68', '1.43', '1.28', '1.09']


In [267]:
pc_final_df = pd.DataFrame(data = pComponents_final, columns = component_col)
print(pc_final_df.shape)
pc_final_df.head()

(1211, 10)


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,2.044488,19.692679,5.910619,-10.398243,-0.712492,-3.640289,-1.501856,0.319966,2.110156,-0.742582
1,2.045330,19.691086,5.909081,-10.405563,-0.709562,-3.565497,-1.416759,0.378697,2.122803,-0.715175
2,2.051763,19.679396,5.899206,-10.465570,-0.686543,-3.013283,-0.795530,0.806163,2.223784,-0.514564
3,2.044010,19.693377,5.910792,-10.393203,-0.714254,-3.673386,-1.538120,0.295580,2.101528,-0.754176
4,2.043780,19.694025,5.911864,-10.391542,-0.715067,-3.704210,-1.573543,0.270412,2.099284,-0.765879


In [268]:
print('Explained variation percentage per principal component: {}'.format(percentage_list))
total_explained_percentage = (sum(pca_final.explained_variance_ratio_)*100)
print('Total percentage of the explained data by',pca_final.n_components,'components is: %.2f' %total_explained_percentage)
print('Percentage of the information that is lost for using',pca_final.n_components,'components is: %.2f' %(100-total_explained_percentage))

Explained variation percentage per principal component: ['34.06', '23.60', '14.98', '5.67', '3.73', '2.54', '1.68', '1.43', '1.28', '1.09']
Total percentage of the explained data by 10 components is: 90.05
Percentage of the information that is lost for using 10 components is: 9.95


In [269]:
mainDf_v3 = mainDf.merge(pc_final_df, left_index=True, right_index=True)
print(mainDf.shape)
print(mainDf_v3.shape)

(1211, 157)
(1211, 167)


## Doing the feature selection again after the addition of the Principal Components

In [270]:
mylist3 = getRowsForRisks(mainDf_v3, risks)
print(len(mylist3))
for i in range(len(mylist3)):
    print("Only",mylist3[i].shape[0],"rows are available for risk",i)

7
Only 87 rows are available for risk 0
Only 148 rows are available for risk 1
Only 262 rows are available for risk 2
Only 184 rows are available for risk 3
Only 98 rows are available for risk 4
Only 65 rows are available for risk 5
Only 59 rows are available for risk 6


In [271]:
for i in range(len(risks)):
    print("-- For the", risks[i],"--")
    getImportantFeatures(mylist3[i], risks[i], risks)

-- For the risk0 --
Picked variable number: 23
                                                 Specs     Score
57   Gross domestic product (GDP), total (2017 PPP ...  4.032056
41             Population ages 65 and older (millions)  3.234697
52   Working poor at PPP$3.20 a day (% of total emp...  2.961307
27                         Inequality in education (%)  2.900850
4    Population with at least some secondary educat...  2.875645
3    Population with at least some secondary educat...  2.794997
17   Foreign direct investment, net inflows (% of GDP)  2.734514
5    Population with at least some secondary educat...  2.687594
152                                                PC7  2.591428
8       Share of seats in parliament (% held by women)  2.518464
43                      Exports and imports (% of GDP)  2.435197
97     Mortality rate, under-5 (per 1,000 live births)  2.434818
54      Youth unemployment rate (female to male ratio)  2.378745
6              Mean years of schooling, fem

# Try generating Polynomial Cross Features

In [280]:
pop_arr = getArrayOfFeatures(mainDf_v3, "population")
print(pop_arr)

['population', 'Total population (millions)', 'Urban population (%)', 'Employment to population ratio (% ages 15 and older)', 'Prevalence of HIV, total (% of population ages 15-49)', 'Rural population with access to safe drinking-water (JMP)', 'Total population with access to safe drinking-water (JMP)', 'Urban population with access to safe drinking-water (JMP)']


In [306]:
cross_df = generateCrossFeat(mainDf, pop_arr)
mainDf_v4 = mainDf.merge(cross_df, left_index=True, right_index=True)
print(mainDf.shape)
print(mainDf_v4.shape)

(1211, 157)
(1211, 202)


In [312]:
mainDf_v4.columns = mainDf_v4.columns.astype(str)

In [313]:
mylist4 = getRowsForRisks(mainDf_v4, risks)
print(len(mylist4))
for i in range(len(mylist4)):
    print("Only",mylist4[i].shape[0],"rows are available for risk",i)

7
Only 87 rows are available for risk 0
Only 148 rows are available for risk 1
Only 262 rows are available for risk 2
Only 184 rows are available for risk 3
Only 98 rows are available for risk 4
Only 65 rows are available for risk 5
Only 59 rows are available for risk 6


In [315]:
for i in range(len(risks)):
    print("-- For the", risks[i],"--")
    getImportantFeatures(mylist4[i], risks[i], risks)

-- For the risk0 --
Picked variable number: 28
                                                 Specs     Score
57   Gross domestic product (GDP), total (2017 PPP ...  4.032056
164                                                 18  3.624336
179                                                 33  3.601730
41             Population ages 65 and older (millions)  3.234697
178                                                 32  3.039628
52   Working poor at PPP$3.20 a day (% of total emp...  2.961307
27                         Inequality in education (%)  2.900850
4    Population with at least some secondary educat...  2.875645
3    Population with at least some secondary educat...  2.794997
17   Foreign direct investment, net inflows (% of GDP)  2.734514
5    Population with at least some secondary educat...  2.687594
180                                                 34  2.531487
8       Share of seats in parliament (% held by women)  2.518464
43                      Exports and imports